In [16]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import json
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse, mae
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import subprocess

In [17]:
def update_user_profiles():
    subprocess.run(["php", "populate_user_profiles.php"], check=True)
    subprocess.run(["php", "populate_menujson.php"], check=True)

In [18]:
def load_user_profiles():
    global user_profiles
    global recipes
    
    # Load user profiles
    with open('users.json', 'r') as f:
        user_profiles = json.load(f)
    
    # Load recipes
    with open('menu.json', 'r') as file:
        recipes = json.load(file)

In [19]:
load_user_profiles()

In [20]:
# Create DataFrame for user ratings
ratings = []
for user in user_profiles:
    user_id = user['user_id']
    for order in user['order_history']:
        ratings.append({'user_id': user_id, 'recipe': order, 'rating': 5})  # Fixed rating for simplicity

ratings_df = pd.DataFrame(ratings)
recipe_df = pd.DataFrame(recipes)

In [21]:
# Prepare data for Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'recipe', 'rating']], reader)

In [22]:
# Split data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2)

In [23]:
# Train the model using SVD
model = SVD()
model.fit(trainset)

In [24]:
# Evaluate the model
predictions = model.test(testset)
print(f"RMSE: {rmse(predictions)}")
print(f"MAE: {mae(predictions)}")

RMSE: 0.0141
RMSE: 0.014052266949775247
MAE:  0.0112
MAE: 0.011204452719823621


In [25]:
# Function to compute content-based similarities
def compute_content_similarity(df):
    df['ingredients_str'] = df['ingredients'].apply(lambda x: ' '.join(x))
    vectorizer = CountVectorizer()
    ingredient_matrix = vectorizer.fit_transform(df['ingredients_str'])
    return cosine_similarity(ingredient_matrix)


In [26]:
# Function to recommend recipes for a user
def recommend_recipes(user_id, num_recommendations=3):
    # Check if user exists
    if user_id not in ratings_df['user_id'].values:
        return f"User ID {user_id} not found."

    user_ratings = ratings_df[ratings_df['user_id'] == user_id]
    user_rated_recipes = user_ratings['recipe'].tolist()

    # 1. Content-Based Recommendations
    content_similarity = compute_content_similarity(recipe_df)

    # 2. Similar Options
    similar_recipes = []
    for order in user_rated_recipes:
        if order in recipe_df['dish_name'].values:
            idx = recipe_df[recipe_df['dish_name'] == order].index[0]
            similarity_scores = list(enumerate(content_similarity[idx]))
            similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

            for i in similarity_scores:
                similar_recipe_name = recipe_df['dish_name'].iloc[i[0]]
                if similar_recipe_name not in user_rated_recipes:
                    similar_recipes.append(similar_recipe_name)
                    break

    # 3. User Past Order Similarity
    user_based_recommendations = []
    for order in user_rated_recipes:
        if order in recipe_df['dish_name'].values:
            idx = recipe_df[recipe_df['dish_name'] == order].index[0]
            similarity_scores = list(enumerate(content_similarity[idx]))
            similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
            recommendations = [recipe_df['dish_name'].iloc[i[0]] for i in similarity_scores if recipe_df['dish_name'].iloc[i[0]] not in user_rated_recipes]
            user_based_recommendations.extend(recommendations)

    # 4. Trending Orders
    trending_recipes = recipe_df.sort_values(by='order_count', ascending=False)['dish_name'].tolist()[:num_recommendations]

    # 5. Combine Recommendations
    combined_recommendations = set(similar_recipes + user_based_recommendations + trending_recipes)
    # 6. Dynamic Suggestions
    recipe_order_counts = ratings_df['recipe'].value_counts().to_dict()
    recommendations_with_scores = []

    for recipe in combined_recommendations:
        est_rating = model.predict(user_id, recipe).est
        est_rating += recipe_order_counts.get(recipe, 0)  # Boost by order count
        
        # Check user preferences
        user_preferences = next((user['preferences'] for user in user_profiles if user['user_id'] == user_id), [])
        if any(pref in recipe for pref in user_preferences):
            est_rating += 1  # Boost for matching preferences

        recommendations_with_scores.append((recipe, est_rating))

    # Sort recommendations based on scores
    recommendations_with_scores = sorted(recommendations_with_scores, key=lambda x: x[1], reverse=True)

    # Return the top recommendations
    return [rec[0] for rec in recommendations_with_scores[:num_recommendations]]


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/recommend', methods=['GET'])
def recommend():
        # Update user profiles before processing recommendations
   
    # return "hello"
    user_id = int(request.args.get('user_id'))
    num_recommendations = int(request.args.get('num_recommendations', 3))
    recommendations = recommend_recipes(user_id, num_recommendations)
    return jsonify(recommendations)

if __name__ == '__main__':
    update_user_profiles()
    load_user_profiles()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://bbb0-104-254-89-53.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Jan/2025 03:27:55] "GET /recommend?user_id=14&num_recommendations=3 HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 03:28:26] "GET /recommend?user_id=14&num_recommendations=6 HTTP/1.1" 200 -
127.0.0.1 - - [27/Jan/2025 03:28:41] "GET /recommend?user_id=14&num_recommendations=3 HTTP/1.1" 200 -
